In [ ]:
# Make a folder for images and download the 4 images uploaded on github

In [5]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Set image directory
image_dir = r'C:\Users\Lacuesta\Desktop\Python\images'

# Load and filter the CSV
df = pd.read_csv("artists.csv")
filtered_df = df[df['paintings'].isin([81, 59, 194, 291])].reset_index(drop=True)

# Create label encoding
label_map = {name: idx for idx, name in enumerate(filtered_df['name'].unique())}
filtered_df['label'] = filtered_df['name'].map(label_map)

# Define dataset
class ArtworkDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.data = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_filename = str(row['paintings']) + '.jpg'
        img_path = os.path.join(self.img_dir, img_filename)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = row['label']
        return image, label

# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create dataset and split
dataset = ArtworkDataset(filtered_df, image_dir, transform=transform)
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2)

# Setup model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(label_map))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")

# Evaluate on validation set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# Print actual vs predicted
print("\nActual vs Predicted Labels:")
for images, labels in val_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    for i in range(len(labels)):
        actual_label = [k for k, v in label_map.items() if v == labels[i].item()][0]
        predicted_label = [k for k, v in label_map.items() if v == predicted[i].item()][0]
        print(f"Actual: {actual_label}, Predicted: {predicted_label}")

Epoch 1, Loss: 3.0338
Epoch 2, Loss: 2.4242
Epoch 3, Loss: 0.8655
Epoch 4, Loss: 2.2894
Epoch 5, Loss: 1.7491
Validation Accuracy: 0.00%

Actual vs Predicted Labels:
Actual: Gustave Courbet, Predicted: Henri de Toulouse-Lautrec
Actual: Rene Magritte, Predicted: Henri de Toulouse-Lautrec
